In [ ]:
from vision.datasets.slr_dataset import SLRDataset
from config import default_config as cfg
import numpy as np

dataset = SLRDataset(root=cfg.datasets[0], transform=None, target_transform=None)
for i in range(len(dataset)):
    try:
        image, boxes, labels = dataset[i]
        print(f"Sample {i} (ID: {dataset.ids[i]}): boxes shape={boxes.shape}, labels shape={labels.shape}")
    except Exception as e:
        print(f"Sample {i} (ID: {dataset.ids[i]}): Error: {str(e)}")

In [6]:
import numpy as np
from numpy import random as np_random
import random  # Python random
import multiprocessing as mp

# Định nghĩa sample_options giống như trong RandomSampleCrop
sample_options = [
    None,
    (0.1, None),
    (0.3, None),
    (0.7, None),
    (0.9, None),
    (None, None),
]

# Hàm test standalone với numpy.random.choice
def test_numpy_standalone():
    print("Testing standalone with numpy.random.choice...")
    for _ in range(10):  # Chạy 10 lần
        try:
            choice = np_random.choice(sample_options)
            print(f"NumPy standalone choice: {choice}")
        except ValueError as e:
            print(f"NumPy standalone error: {e}")
            break

# Hàm test standalone với Python random.choice
def test_python_standalone():
    print("\nTesting standalone with Python random.choice...")
    for _ in range(10):  # Chạy 10 lần
        try:
            choice = random.choice(sample_options)
            print(f"Python standalone choice: {choice}")
        except Exception as e:
            print(f"Python standalone error: {e}")
            break

# Hàm test multiprocessing với numpy.random.choice
def test_numpy_multiprocessing(q):
    process_name = mp.current_process().name
    print(f"\nTesting multiprocessing with numpy.random.choice in {process_name}...")
    for _ in range(5):  # Chạy 5 lần trong mỗi process
        try:
            choice = np_random.choice(sample_options)
            print(f"NumPy multiprocessing choice in {process_name}: {choice}")
            q.put(choice)
        except ValueError as e:
            print(f"NumPy multiprocessing error in {process_name}: {e}")
            q.put(str(e))
            break

# Hàm test multiprocessing với Python random.choice
def test_python_multiprocessing(q):
    process_name = mp.current_process().name
    print(f"\nTesting multiprocessing with Python random.choice in {process_name}...")
    for _ in range(5):  # Chạy 5 lần trong mỗi process
        try:
            choice = random.choice(sample_options)
            print(f"Python multiprocessing choice in {process_name}: {choice}")
            q.put(choice)
        except Exception as e:
            print(f"Python multiprocessing error in {process_name}: {e}")
            q.put(str(e))
            break

if __name__ == "__main__":
    # Test standalone
    test_numpy_standalone()
    test_python_standalone()

    # Test multiprocessing
    print("\nStarting multiprocessing tests...")
    queue = mp.Queue()
    processes = []
    num_workers = 2  # Mô phỏng DataLoader với 2 workers

    # Test NumPy random.choice trong multiprocessing
    for _ in range(num_workers):
        p = mp.Process(target=test_numpy_multiprocessing, args=(queue,))
        processes.append(p)
        p.start()

    # Thu thập kết quả từ NumPy test
    numpy_results = [queue.get() for _ in range(num_workers * 5)]  # Lấy đủ kết quả
    for p in processes:
        p.join()

    # Reset processes để test Python random.choice
    processes = []
    for _ in range(num_workers):
        p = mp.Process(target=test_python_multiprocessing, args=(queue,))
        processes.append(p)
        p.start()

    # Thu thập kết quả từ Python test
    python_results = [queue.get() for _ in range(num_workers * 5)]  # Lấy đủ kết quả
    for p in processes:
        p.join()

    # In kết quả
    print("\nResults from NumPy multiprocessing:")
    for result in numpy_results:
        print(result)

    print("\nResults from Python multiprocessing:")
    for result in python_results:
        print(result)

Testing standalone with numpy.random.choice...
NumPy standalone error: setting an array element with a sequence. The requested array has an inhomogeneous shape after 1 dimensions. The detected shape was (6,) + inhomogeneous part.

Testing standalone with Python random.choice...
Python standalone choice: (0.1, None)
Python standalone choice: (0.7, None)
Python standalone choice: None
Python standalone choice: (0.3, None)
Python standalone choice: (0.9, None)
Python standalone choice: (0.3, None)
Python standalone choice: (0.9, None)
Python standalone choice: (0.3, None)
Python standalone choice: (0.9, None)
Python standalone choice: (0.7, None)

Starting multiprocessing tests...

Testing multiprocessing with numpy.random.choice in Process-3...
NumPy multiprocessing error in Process-3: setting an array element with a sequence. The requested array has an inhomogeneous shape after 1 dimensions. The detected shape was (6,) + inhomogeneous part.

Testing multiprocessing with numpy.random.cho

KeyboardInterrupt: 